# 1. Load and Inspect Data

In [0]:
df = spark.table ("samples.nyctaxi.trips")
df.printSchema()
df.show(5)
display(df)

# Check for Missing Values

In [0]:
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()


# Check for duplicates

In [0]:
df.count(), df.dropDuplicates().count()


# Check for outliers

In [0]:
from pyspark.sql.functions import expr

# Example for fare_amount
quantiles = df.approxQuantile("fare_amount", [0.25, 0.75], 0.05)
Q1, Q3 = quantiles
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_outliers = df.filter((col("fare_amount") < lower_bound) | (col("fare_amount") > upper_bound))
df_outliers.select("fare_amount").show()


# convert to pandas on spark for profiling

In [0]:
import pyspark.pandas as ps

psdf = df.to_pandas_on_spark()
psdf.info()
psdf.describe()


# Group and Aggregate

In [0]:
orderby = df.groupBy("pickup_zip").agg({"fare_amount": "avg"}).orderBy("avg(fare_amount)",ascending=False)

display(orderby)

In [0]:
display(df.select("trip_distance","fare_amount"))

Databricks visualization. Run in Databricks to view.

Here's a small change
